In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://www.bbc.co.uk/sport/football/64818881"
response = requests.get(url)


In [3]:
response.status_code

404

In [53]:
# soup.find_all('article')

In [54]:
full_article = ''
for el in soup.find_all('article'):
     full_article += el.get_text().strip()

In [55]:
full_article

'Greece train crash: 57 people confirmed dead as public anger growsPublished2 hours agoShareclose panelShare pageCopy linkAbout sharingImage source, ReutersImage caption, Remains of destroyed train carriage being removed by crane on Thursday morningBy Nick Beake in Larissa & Kathryn Armstrong in LondonBBC NewsThe death toll from Tuesday\'s train crash in Greece has increased to 57, a coroner has told the BBC. Eleni Zaggelidou, one of ten coroners working on the investigation, said DNA had been taken from 57 intact bodies.Meanwhile, a government minister said austerity during Greece\'s economic crisis in the 2000s contributed to a lack of investment in the railways.Rail workers held a one-day strike on Thursday following the disaster, blaming government neglect. More than 2,000 people protested for a second day in Athens and Thessaloniki, shocked by Tuesday\'s disaster near the city of Larissa."We are angry at the company, at the government and past governments that did nothing to impro

In [17]:
import os
import shelve
import pickle
from collections import Counter
import math

In [6]:
# common.py
from bs4 import BeautifulSoup
import requests
def extract_sitemaps(sitemap_url):
    res = requests.get(sitemap_url)
    if res.status_code != 200:
        return []

    soup = BeautifulSoup(res.text, 'xml')

    sitemap_tags = soup.find_all('sitemap')

    sitemaps = []
    for sitemap in sitemap_tags:
        loc = sitemap.find('loc')
        sitemaps.append(loc.text)
    return sitemaps




def extract_urls_from_sitemap(sitemap_url):
    res = requests.get(sitemap_url)
    if res.status_code != 200:
        return []

    soup = BeautifulSoup(res.text, 'xml')

    urls = []
    url_tags = soup.find_all('url')
    for url in url_tags:
        loc = url.find('loc')
        urls.append(loc.text)

    return urls

In [33]:
# //helper.py
import requests
from bs4 import BeautifulSoup


def extract_text_from_url(url):
    res = requests.get(url)
    if res.status_code != 200:
        return ''

    soup = BeautifulSoup(res.text, 'html.parser')
    h1 = soup.find('h1')
    if h1 is None:
        return ''

    main_content = soup.find_all('article')
    chunks = []
    chunks.append(h1.get_text().strip())
    for p in main_content[:2]:
        chunks.append(p.get_text().strip())

    return '\n'.join(chunks)

In [34]:
def crawl(visited_urls):
    sitemap_url = 'https://www.bbc.com/sitemaps/https-index-com-news.xml'
    sitemaps = extract_sitemaps(sitemap_url)
    for sitemap in sitemaps[1:101]:
        urls = extract_urls_from_sitemap(sitemap)
        for url in urls:
            if url not in visited_urls:
#                 visited_urls.add(url)
                text = extract_text_from_url(url)
                print(text)

In [35]:
import os

In [36]:
visited_urls_file = os.path.join(os.getcwd(), 'db', 'visited_urls.db')